In [6]:
import numpy as np
import tensorflow as tf
from keras.models import Sequential, Model
from keras.layers import Input, Conv2D, MaxPooling2D, Flatten, Dense, Dropout, Reshape, BatchNormalization, LeakyReLU
from keras.optimizers import Adam
from keras.callbacks import ModelCheckpoint
from keras.regularizers import l2
from tensorflow.keras.losses import CategoricalCrossentropy
from tensorflow.keras.utils import Sequence
from keras.applications import VGG16


In [7]:
# Define the dice_coefficient function
def dice_coefficient(y_true, y_pred):
    smooth = 1e-5
    intersection = tf.reduce_sum(y_true * y_pred)
    union = tf.reduce_sum(y_true) + tf.reduce_sum(y_pred)
    return (2.0 * intersection + smooth) / (union + smooth)

In [8]:

# Define la clase del generador de datos
class DataGenerator(Sequence):
    def __init__(self, imagenes, heatmaps, batch_size=32):
        self.imagenes = imagenes
        self.heatmaps = heatmaps
        self.batch_size = batch_size
        self.indexes = np.arange(len(self.imagenes))

    def __len__(self):
        return int(np.ceil(len(self.imagenes) / self.batch_size))

    def __getitem__(self, index):
        start = index * self.batch_size
        end = (index + 1) * self.batch_size
        batch_x = self.imagenes[start:end]
        batch_y = self.heatmaps[start:end]
        return batch_x, batch_y

In [9]:
# Rutas de los archivos
imagenes_train_path = r"D:\Tec\7mo Smestre\Inteligencia Artificial II\Reto\Datos\Arrays Heatmaps\train\trainimagenes_originales_train.npy"
imagenes_val_path = r"D:\Tec\7mo Smestre\Inteligencia Artificial II\Reto\Datos\Arrays Heatmaps\val\valimagenes_originales_val.npy"
imagenes_test_path = r"D:\Tec\7mo Smestre\Inteligencia Artificial II\Reto\Datos\Arrays Heatmaps\test\trainimagenes_originales_test.npy"
heatmaps_train_path = r"D:\Tec\7mo Smestre\Inteligencia Artificial II\Reto\Datos\Arrays Heatmaps\train\trainheatmaps_train.npy"
heatmaps_val_path = r"D:\Tec\7mo Smestre\Inteligencia Artificial II\Reto\Datos\Arrays Heatmaps\val\valheatmaps_val.npy"
heatmaps_test_path = r"D:\Tec\7mo Smestre\Inteligencia Artificial II\Reto\Datos\Arrays Heatmaps\test\trainheatmaps_test.npy"

# Cargar los archivos
imagenes_train = np.load(imagenes_train_path)
heatmaps_train = np.load(heatmaps_train_path)
imagenes_val = np.load(imagenes_val_path)
heatmaps_val = np.load(heatmaps_val_path)
imagenes_test = np.load(imagenes_test_path)
heatmaps_test = np.load(heatmaps_test_path)

# Convertir uint8 tensor a float32 y normalizar imágenes
imagenes_train = imagenes_train.astype('float32') / 255.0
imagenes_val = imagenes_val.astype('float32') / 255.0
imagenes_test = imagenes_test.astype('float32') / 255.0

In [10]:
from keras.applications import VGG16
from keras.models import Model
from keras.layers import Input, Conv2D, MaxPooling2D, Flatten, Dense, Dropout, Reshape, BatchNormalization
from keras.optimizers import Adam
from keras.callbacks import ModelCheckpoint
from keras.regularizers import l2
from keras.metrics import categorical_crossentropy
from keras.losses import CategoricalCrossentropy
from keras import backend as K
import numpy as np

# Cargar el modelo VGG16 preentrenado
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(112, 112, 3))

# Congelar las capas del modelo base
for layer in base_model.layers:
    layer.trainable = False

# Agregar capas personalizadas al modelo
x = base_model.output
x = Conv2D(512, (3, 3), padding='same', activation='relu')(x)
x = Conv2D(512, (3, 3), padding='same', activation='relu')(x)
x = Conv2D(512, (3, 3), padding='same', activation='relu')(x)
x = MaxPooling2D((2, 2))(x)

x = Dropout(0.5)(x)

# Regularización L2
x = Flatten()(x)
x = Dense(2048, activation='relu', kernel_regularizer=l2(0.0001))(x)
x = Dropout(0.5)(x)
x = Dense(2048, activation='relu', kernel_regularizer=l2(0.0001))(x)
x = Dropout(0.5)(x)
x = Dense(112 * 112 * 7, activation='sigmoid')(x)
output_layer = Reshape((112, 112, 7))(x)

# Crear el modelo completo
model = Model(inputs=base_model.input, outputs=output_layer)

# Configurar el checkpoint para guardar el mejor modelo
checkpoint = ModelCheckpoint("mejor_modelo.h5", monitor='val_dice_coefficient', verbose=1, save_best_only=True, mode='max')

# Compilar el modelo
model.compile(optimizer=Adam(), loss=CategoricalCrossentropy(), metrics=[dice_coefficient])

# Crear instancias del generador de datos para entrenamiento y validación
train_generator = DataGenerator(imagenes_train, heatmaps_train, batch_size=16)
val_generator = DataGenerator(imagenes_val, heatmaps_val, batch_size=16)

# Entrenar el modelo
history = model.fit(train_generator, epochs=30, validation_data=val_generator, callbacks=[checkpoint])

# Evaluar el modelo en datos de prueba
test_loss, test_dice_score = model.evaluate(imagenes_test, heatmaps_test)
print(f'Loss en datos de prueba: {test_loss}')
print(f'DICE SCORE en datos de prueba: {test_dice_score}')

Epoch 1/30


ResourceExhaustedError: Graph execution error:

Detected at node 'gradient_tape/model_1/dense_5/MatMul/MatMul_1' defined at (most recent call last):
    File "c:\Users\vhae1\anaconda3\envs\tf\lib\runpy.py", line 197, in _run_module_as_main
      return _run_code(code, main_globals, None,
    File "c:\Users\vhae1\anaconda3\envs\tf\lib\runpy.py", line 87, in _run_code
      exec(code, run_globals)
    File "c:\Users\vhae1\anaconda3\envs\tf\lib\site-packages\ipykernel_launcher.py", line 17, in <module>
      app.launch_new_instance()
    File "c:\Users\vhae1\anaconda3\envs\tf\lib\site-packages\traitlets\config\application.py", line 1053, in launch_instance
      app.start()
    File "c:\Users\vhae1\anaconda3\envs\tf\lib\site-packages\ipykernel\kernelapp.py", line 737, in start
      self.io_loop.start()
    File "c:\Users\vhae1\anaconda3\envs\tf\lib\site-packages\tornado\platform\asyncio.py", line 215, in start
      self.asyncio_loop.run_forever()
    File "c:\Users\vhae1\anaconda3\envs\